In [23]:
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import ast
from typing import List, Tuple

from collections import defaultdict
from typing import List                                                                                                         
from tqdm import tqdm

# 30 sec

In [24]:
from Enum.EnergyRange import EnergyRange
from Enum.Flavour import Flavour

In [25]:
sys.path.append('/groups/icecube/cyan/Utils')
from PlotUtils import setMplParam, getColour, getHistoParam 
# getHistoParam:
# Nbins, binwidth, bins, counts, bin_centers  = 
from DB_lister import list_content, list_tables
from ExternalFunctions import nice_string_output, add_text_to_ax
setMplParam()

In [26]:
csvs_medium_mu_vs_e = {
    r"Track vs. Cascade, E$\in$(10TeV,1PeV), train(250m), val(250m)": "/lustre/hpc/icecube/cyan/factory/IceCubeTransformer/predictions/20250410/model_20250330_203131/112439/predictions_epoch_012_val_loss_0.0984.csv"
}

In [30]:
def get_track_cascade_logits(df: pd.DataFrame) -> np.ndarray:
    def safe_parse(x):
        return ast.literal_eval(x) if isinstance(x, str) else x
    
    df["logits"] = df["logits"].apply(safe_parse)
    return df["logits"].apply(lambda x: x[0]).to_numpy()

In [43]:
def plot_csvs(csvs:dict, is_magnifier_on:bool):
    for id, csv in csvs.items():
        df = pd.read_csv(csv)
        print(df[['target_class','pred_class','target_one_hot_pid','logits']].head())
        # print(get_track_cascade_logits(df))   

In [44]:
plot_csvs(csvs_medium_mu_vs_e, False)

   target_class  pred_class target_one_hot_pid  \
0             1           1             [0, 1]   
1             1           0             [0, 1]   
2             1           1             [0, 1]   
3             1           1             [0, 1]   
4             1           1             [0, 1]   

                                      logits  
0   [0.0325930193066597, 0.9659360647201538]  
1    [0.962875485420227, 0.0307389497756958]  
2   [0.08216621726751328, 0.917912483215332]  
3  [0.06982182711362839, 0.9284912347793579]  
4  [0.03694513440132141, 0.9634585976600647]  
